# Fake News Generator and Detector

## Introduction
This project demonstrates a dual-purpose Natural Language Processing (NLP) application that uses transformer-based models to tackle the challenge of misinformation. The system consists of two main components: a **Fake News Generator**, which utilizes GPT-2 to produce realistic-looking news articles based on a user-provided prompt, and a **Fake News Detector**, which employs BERT to classify given news content as either fake or real. By combining generative and discriminative models, this project offers a hands-on exploration of language model capabilities and the implications of AI-generated content.

## Objective
The main objective is to build a user-friendly application capable of generating synthetic news text and detecting the authenticity of a given article or statement. This project highlights both the creative power of transformer models and their utility in identifying misinformation.

## Tools and Technologies
- **Python**
- **PyTorch**
- **Hugging Face Transformers (GPT-2 & BERT)**
- **Gradio** for the web interface

## Project Flow
1. Load and configure pre-trained models from Hugging Face.
2. Use GPT-2 to generate news content based on a prompt.
3. Use BERT for binary classification of input news as fake or real.
4. Build a simple interactive UI using Gradio to access both functionalities.

---



In [ ]:
!pip install -q transformers torch gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.7 MB/s eta 0:00:00


In [ ]:
# ✅ Imports
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr

# ✅ Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load Models
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
).to(device)

# ✅ Generate Fake News
def generate_fake_news(prompt):
    inputs = gpt2_tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = gpt2_model.generate(
        inputs,
        max_length=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        early_stopping=True
    )
    generated_text = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# ✅ Detect Fake or Real News
def detect_news(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()
    label = "🟥 Fake News" if predicted_class == 0 else "🟩 Real News"
    return f"{label} (Confidence: {confidence:.2f})"

# ✅ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📰 Fake News Generator & Detector (GPT-2 + BERT)")
    with gr.Tab("🛠️ Generate Fake News"):
        with gr.Row():
            input_text = gr.Textbox(label="Enter a News Headline or Prompt", placeholder="e.g. A mysterious object was spotted in the sky...", lines=2)
        generate_btn = gr.Button("Generate")
        output_text = gr.Textbox(label="Generated News Article")
        generate_btn.click(generate_fake_news, inputs=input_text, outputs=output_text)

    with gr.Tab("🔍 Detect Fake or Real"):
        with gr.Row():
            detect_input = gr.Textbox(label="Enter a News Article or Statement", placeholder="Paste a paragraph to detect if it's fake or real...", lines=5)
        detect_btn = gr.Button("Detect")
        detect_output = gr.Textbox(label="Detection Result")
        detect_btn.click(detect_news, inputs=detect_input, outputs=detect_output)

# ✅ Launch
demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60b961ec3376856736.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
